# Generate tiles for fine resolution land relief parameterization

This script is dedicated to generating tile that can be used for fine resolution land relief parameterization. In general, they are three types of tile.

1. regional tile

Regional tile has an extend distance of 3392*30=101,760km (pixel x resolution). Due to the factor of 2, which in 960m resolution the extend pixels would be 106. 

2. local tile

Local tile has an extend distance of 64*30=1.92km (pixel x resolution). Due to the factor of 2, which in 240m resolution the extend pixels would be 8

3. final tile
final tile is used to crop the parameters before saving. It has an extend distance of 960m. It removes the border effect and the tile would be 4 pixels overlap to make sure there'd be no line in the map. 

- irregular tile

There is a set of tile which is classified as irregular tiles. The irregular tiles were either missing out from the equi7 tiling system or across 180th meridians. The detail is explained in the paper. Due to its characteristics, the tile has the same boundary of regional, local and final tile. The reason is these irregular tiles would service at the bottom of the tile so the border effect will be covered by the regular tiles.

## Part 1: Setup

In [4]:
import os
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon,mapping,box
from shapely import segmentize
import pickle

def find_equi_7_proj(epsg4326_bound,equi_7_proj):
    # Extract coordinates from bounding box
    xmin, ymin, xmax, ymax = epsg4326_bound

    # Define coordinates for the polygon
    polygon_coords = [(xmin, ymin), (xmax, ymin), (xmax, ymax),(xmin, ymax)]

    # Create polygon
    polygon = Polygon(polygon_coords)

    # Create a GeoDataFrame with a single row representing the bounding box
    gdf = gpd.GeoDataFrame(geometry=[Polygon(polygon_coords)])
    # Set CRS for the GeoDataFrame
    gdf.crs = 'EPSG:4326'
    # Check the resulting GeoDataFrame
    gdf.geometry = segmentize(gdf.geometry,max_segment_length=0.000277778)

    return gdf.to_crs(equi_7_proj).geometry.bounds.values[0]

def find_epsg4326_proj(equi7_bound,equi_7_proj):
    # Extract coordinates from bounding box
    xmin, ymin, xmax, ymax = equi7_bound

    # Define coordinates for the polygon
    polygon_coords = [(xmin, ymin), (xmax, ymin), (xmax, ymax),(xmin, ymax)]

    # Create polygon
    polygon = Polygon(polygon_coords)

    # Create a GeoDataFrame with a single row representing the bounding box
    gdf = gpd.GeoDataFrame(geometry=[Polygon(polygon_coords)])
    # Set CRS for the GeoDataFrame
    gdf.crs = equi_7_proj
    gdf.geometry = segmentize(gdf.geometry,max_segment_length=30)
    return gdf.to_crs('EPSG:4326')

equi7_ind={'AF':'+proj=aeqd +lat_0=8.5 +lon_0=21.5 +x_0=5621452.01998 +y_0=5990638.42298 +datum=WGS84 +units=m +no_defs',
'AN':'+proj=aeqd +lat_0=-90 +lon_0=0 +x_0=3714266.97719 +y_0=3402016.50625 +datum=WGS84 +units=m +no_defs',
'AS':'+proj=aeqd +lat_0=47 +lon_0=94 +x_0=4340913.84808 +y_0=4812712.92347 +datum=WGS84 +units=m +no_defs',
'EU':'+proj=aeqd +lat_0=53 +lon_0=24 +x_0=5837287.81977 +y_0=2121415.69617 +datum=WGS84 +units=m +no_defs',
'NA':'+proj=aeqd +lat_0=52 +lon_0=-97.5 +x_0=8264722.17686 +y_0=4867518.35323 +datum=WGS84 +units=m +no_defs',
'OC':'+proj=aeqd +lat_0=-19.5 +lon_0=131.5 +x_0=6988408.5356 +y_0=7654884.53733 +datum=WGS84 +units=m +no_defs',
'SA':'+proj=aeqd +lat_0=-14 +lon_0=-60.5 +x_0=7257179.23559 +y_0=5592024.44605 +datum=WGS84 +units=m +no_defs'}

conts = ['AF',
'AN',
'AS',
'EU',
'NA',
'OC',
'SA']



/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.11.1-CAPI-1.17.1). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_8100/3041233406.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling 

## Part 2: Generate tiles in a for-loop

- loop1: continent
- loop2: tiles

The irregular tiles have their own loops and at the final the two lists will be merged into one.

The Equi7Grid tile can be downloaded from this GitHub repository (https://github.com/TUW-GEO/Equi7Grid)

In [26]:
args=[]
rls_padding=3392     #3392*30=101760 is equivalent to 106 pixel in 960m resolution
ls_padding=64        #64*30=960 is equivalent to 1 pixel in 960m resolution
for cont in conts:
    equi_7_proj=equi7_ind[cont]
    proj_dir = f'Equi7Grid/src/equi7grid/grids/{cont}/PROJ/'
    for i in [i for i in os.scandir(proj_dir) if 'T6' in i.name and 'shp' in i.name]:
        equi7_grid = gpd.read_file(i.path)
        equi7_grid = equi7_grid[equi7_grid['COVERSLAND']==1]
        epsg4326_grid = equi7_grid.to_crs(epsg=4326)
        for tile_id in range(len(equi7_grid)):
            equi7_bound = np.array(equi7_grid.bounds.iloc[tile_id])
            epsg4326_bound = np.array(epsg4326_grid.bounds.iloc[tile_id])
            tile_name = equi7_grid.SHORTNAME.iloc[tile_id].split('_')[-1] + '_' + equi7_grid.TILE.iloc[tile_id] 
            tile = equi7_grid.TILE.iloc[tile_id]
            if tile_name.startswith('AN'):
                continue
            if abs(epsg4326_bound[0] - epsg4326_bound[2]) >150 : ##AS
                continue

            equi7_bounds = equi7_grid[equi7_grid['TILE'] == tile].geometry.bounds
            equi7_bounds = np.array(equi7_bounds)[0]
            equi7_bounds_final = np.array([equi7_bounds[0]-960
                                           ,equi7_bounds[1]-960 ,equi7_bounds[2]+960 ,equi7_bounds[3]+960])
            equi7_bounds_rls = np.array([equi7_bounds[0]-rls_padding*30
                                           ,equi7_bounds[1]-rls_padding*30 ,equi7_bounds[2]+rls_padding*30 ,equi7_bounds[3]+rls_padding*30])
            equi7_bounds_ls = np.array([equi7_bounds[0]-ls_padding*30
                                           ,equi7_bounds[1]-ls_padding*30 ,equi7_bounds[2]+ls_padding*30 ,equi7_bounds[3]+ls_padding*30])

            epsg4326_bounds_final = find_epsg4326_proj(equi7_bounds_final,equi_7_proj).geometry.bounds.values[0]
            epsg4326_bounds_rls = find_epsg4326_proj(equi7_bounds_rls,equi_7_proj).geometry.bounds.values[0]
            epsg4326_bounds_ls = find_epsg4326_proj(equi7_bounds_ls,equi_7_proj).geometry.bounds.values[0]
            args.append((equi7_bounds_final,equi7_bounds_rls,epsg4326_bounds_rls,equi7_bounds_ls,epsg4326_bounds_ls,tile_name,equi_7_proj))

# russia-us
epsg4326_bounds_chuckotka = np.array([-180,50.696,-168.3,83.3])
epsg4326_bounds_chuckotka2 = np.array([159.657,58.148,180,73.442])
epsg4326_bounds_agattu = np.array([172.3292,53.2292,176.3044,52.0630])
# pacific islands
epsg4326_bounds_pukapuka = np.array([-163.621,-13.863,-162.621,-12.863])
epsg4326_bounds_plamerston = np.array([-163.6532,-18.5411,-162.6532,-17.5411])
epsg4326_bounds_futuna = np.array([-178.60509,-14.91163,-177.60509,-13.91163])
epsg4326_bounds_vahen = np.array([-176.13766,-16.10176,-175.13766,-15.10176])
epsg4326_bounds_uvea = np.array([-175.70106,-12.78761,-176.70106,-13.78761])
epsg4326_bounds_tonga = np.array([-175.6306,-21.0206,-173.6306,-19.0206])
epsg4326_bounds_hunga = np.array([-175.99074,-21.14193,-174.99074,-20.14193])
epsg4326_bounds_tuvana = np.array([-179.29184,-21.52716,-178.29184,-20.52716])
epsg4326_bounds_tonga_atoll = np.array([-179.5027,-24.2846,-178.5027,-23.2846])
epsg4326_bounds_kermadec = np.array([-179.2019,-30.7827,-177.2019,-28.7827])
epsg4326_bounds_moutere = np.array([168.66513,-53.04048,169.66513,-52.04048])
epsg4326_bounds_newzealand_north = np.array([172.122,-41.794,180,-33.825])
epsg4326_bounds_okinotori = np.array([136.177838,20.322660,135.977838,20.522660])
epsg4326_bounds_yap = np.array([140.88595,7.63592,139.88595,8.63592])
epsg4326_bounds_yap2 = np.array([142.6412,6.4175,148.079,9.494])
epsg4326_bounds_luf = np.array([144.56444,-2.02959,145.56444,-1.02959])
epsg4326_bounds_anewetak = np.array([161.7339,11.0093,162.7339,12.0093])
epsg4326_bounds_tuvalu = np.array([176.7941,-8.6144,180,-6.9762])
epsg4326_bounds_niulao = np.array([177.24250,-6.20757,177.44250,-6.00757])
epsg4326_bounds_viti_levu=np.array([176.303,-18.991, 180,-16.026])
epsg4326_bounds_taomaroa = np.array([176.72487,-2.73982,176.92487,-2.53982])
epsg4326_bounds_nouvelle_caledonie=np.array([157.8054,-22.2446,160.0386,-18.5696])
epsg4326_bounds_nikunau = np.array([175.93294,-1.45133,176.54619,-1.26088])
epsg4326_bounds_majel = np.array([170.00179,12.13888,170.20179,12.33888])

edges = {'chuckotka' : np.array([-180,50.696,-168.3,83.3]),
'chuckotka2' : np.array([159.657,58.148,180,73.442]),
'agattu' : np.array([172.3292,52.0630,176.3044,53.2292]),
'pukapuka' : np.array([-163.621,-13.863,-162.621,-12.863]),
'plamerston' : np.array([-163.6532,-18.5411,-162.6532,-17.5411]),
'futuna' : np.array([-178.60509,-14.91163,-177.60509,-13.91163]),
'vahen' : np.array([-176.13766,-16.10176,-175.13766,-15.10176]),
'uvea' : np.array([-176.70106,-13.78761,-175.70106,-12.78761]),
'tonga' : np.array([-175.6306,-21.0206,-173.6306,-19.0206]),
'hunga' : np.array([-175.99074,-21.14193,-174.99074,-20.14193]),
'tuvana' : np.array([-179.29184,-21.52716,-178.29184,-20.52716]),
'tonga.atoll' : np.array([-179.5027,-24.2846,-178.5027,-23.2846]),
'kermadec' : np.array([-179.2019,-30.7827,-177.2019,-28.7827]),
'moutere' : np.array([168.66513,-53.04048,169.66513,-52.04048]),
'newzealand.north' : np.array([172.122,-41.794,180,-33.825]),
'okinotori' : np.array([135.977838,20.322660,136.177838,20.522660]),
'yap' : np.array([139.88595,7.63592,140.88595,8.63592]),
'yap2' : np.array([142.6412,6.4175,148.079,9.494]),
'luf' : np.array([144.56444,-2.02959,145.56444,-1.02959]),
'anewetak' : np.array([161.7339,11.0093,162.7339,12.0093]),
'tuvalu' : np.array([176.7941,-8.6144,180,-6.9762]),
'niulao' : np.array([177.24250,-6.20757,177.44250,-6.00757]),
'viti.levu' :np.array([176.303,-18.991, 180,-16.026]),
'taomaroa' : np.array([176.72487,-2.73982,176.92487,-2.53982]),
'nouvelle.caledonie':np.array([157.8054,-22.2446,160.0386,-18.5696]),
'nikunau' : np.array([175.93294,-1.45133,176.54619,-1.26088]),
'majel' : np.array([170.00179,12.13888,170.20179,12.33888]),
'pukapuka': np.array([-163.621,-13.863,-162.621,-12.863]),
'muru' : np.array([169.750032,-11.711507,169.950032,-11.511507]),
'fututaka': np.array([170.089185,-12.009242,170.289185,-11.809242])}

args_special=[]
for i in ['chuckotka','chuckotka2','agattu']:
    epsg4326_bounds = edges[i]
    equi7_bounds = find_equi_7_proj(epsg4326_bounds, equi7_ind['AS'])
    args_special.append((equi7_bounds,equi7_bounds,epsg4326_bounds,equi7_bounds,epsg4326_bounds,f'extracont_{i}',equi7_ind['AS']))
    
for i in ['pukapuka','plamerston','futuna','vahen','uvea','tonga','hunga','tuvana','tonga.atoll','kermadec','moutere','newzealand.north','okinotori','yap','yap2','luf','anewetak','tuvalu','niulao','viti.levu','taomaroa','nouvelle.caledonie','nikunau','majel','muru','fututaka']:
    epsg4326_bounds = edges[i]
    equi7_bounds = find_equi_7_proj(epsg4326_bounds, equi7_ind['OC'])
    args_special.append((equi7_bounds,equi7_bounds,epsg4326_bounds,equi7_bounds,epsg4326_bounds,f'extracont_{i}',equi7_ind['OC']))

In [30]:
# save the tile list in pickle
with open('equi7_tiles', 'wb') as f:
    pickle.dump(args+args_special, f)

In [5]:
# total number of tiles
len(args+args_special)

NameError: name 'args' is not defined